In [ ]:
# Load libraries

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import time

In [ ]:
# Specify the years in the list for which the news has to be extracted
year = []
for i in range(2000,2001):
   year.append(i)
year

In [ ]:
url_list = []
month =[1,2,3,4,5,6,7,8,9,10,11,12]
base_url = "https://api.nytimes.com/svc/archive/v1/"
api_key =<your api key here> 

#Get URLs for every month of the years specified in the years list
for y in year:
    for m in month:
        full_url = base_url + str(y) + "/" + str(m) + ".json?api-key=" + api_key 
        #print (full_url)
        url_list.append(full_url)
url_list

In [ ]:
#create an empty dataframe
main_df = pd.DataFrame()

for url in url_list:
    print (url)
    df_json = pd.read_json(url, orient='records') 
    details = df_json.iloc[0]['response']
    df_json1 = pd.DataFrame(details)
    df_json1 = df_json1[['abstract','headline','pub_date']]
    result = main_df.append(df_json1)
    print(result.tail())
    #keeping a wait time of 4 seconds so that requests get processed properly without http error
    time.sleep(4)
       

In [ ]:
result.head(20)

In [ ]:
#Extract only the year from the publication date column
result["pub_date"]= result["pub_date"].str.split("-", n = 1, expand = True) 
result.head(20)

In [ ]:
#delete the empty rows (with none value in the abstract column)
result.dropna(axis=0, how='any')

In [ ]:
#Delete all records containing main:corrections in the headline column
result[result.headline.str.contains('Corrections')]
result

In [ ]:
#result1.replace(regex="'", value='')
#result1

In [ ]:
#Loading all the above news data in the sqlite database

# Import SQL Alchemy
from sqlalchemy import create_engine
engine = create_engine("sqlite:///NYTimesNewsDB.sqlite")
conn = engine.connect()

In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# Making sure that the table doesnt exist
engine.execute("DROP TABLE IF EXISTS NYTimes_News_Table")
engine.table_names()

In [ ]:
#Loading the data from pandas dataframe to the database table
result.to_sql(name='NYTimes_News_Table', con=engine, if_exists='append', index=True)
engine.table_names()

In [ ]:
# Executing query to verify the data got stored in the DB table
result_set = engine.execute("SELECT * From NYTimes_News_Table")
for record in result_set:
    print(record)